# Working with IDC cohorts

In this example we show how a cohort manifest defined using the IDC Portal can be used to download the data to a cloud VM instance.

This example was prepared using a pre-release version of IDC Portal, and it has not yet been tested using the publicly released version.

To proceed with the cells below you will need to upload your manifest to the connected runtime file system, and replace `<MANIFEST_LOCAL_PATH>` in the examples below with the actual path to the uploaded manifest.

In [ ]:
!head <MANIFEST_LOCAL_FILE_PATH>


You can import IDC cohort manifest in CSV format as any other CSV file, but make sure you check the header to confirm how many lines need to be ignored. The header length may change leading to the public release of the portal.

In [ ]:
import pandas as pd

def cohort_as_df(manifest_filename):
  df = pd.read_csv(manifest_filename, header=5)
  return df

cohort_df = cohort_as_df("<MANIFEST_LOCAL_FILE_PATH>")

The manifest will contain a Google Storage URI (`gs://`) for each of the files corresponding to the individual DICOM instances. Here we save the column containing those URIs to enable download.

In [ ]:
print(cohort_df["gcs_path"])
cohort_df["gcs_path"].to_csv("gcs_paths.txt", header=False, index=False)

0     gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.1451...
1     gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.1451...
2     gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.1451...
3     gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.1451...
4     gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.1451...
5     gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.1451...
6     gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.1451...
7     gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.1451...
8     gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.1451...
9     gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.1451...
10    gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.1451...
11    gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.1451...
12    gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.1451...
13    gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.1451...
14    gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.1451...
15    gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.1451...
16    gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.1451...
17    gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.

In [ ]:
!head /content/gcs_paths.txt

gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.299257181285137013436464151874/1.2.276.0.7230010.3.1.3.0.27811.1553311167.110292/1.2.276.0.7230010.3.1.4.0.27811.1553311167.110290.dcm#1592618476761048
gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.224985459390356936417021464571/1.2.276.0.7230010.3.1.3.0.57863.1553343874.661503/1.2.276.0.7230010.3.1.4.0.57863.1553343874.661501.dcm#1592617650034487
gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.299257181285137013436464151874/1.3.6.1.4.1.14519.5.2.1.6279.6001.251204142823446211001577555181/1.3.6.1.4.1.14519.5.2.1.6279.6001.216241658981991335900886782934.dcm#1592618477049520
gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.195975724868929317649402600442/1.3.6.1.4.1.14519.5.2.1.6279.6001.229343399861261429237689489892/1.3.6.1.4.1.14519.5.2.1.6279.6001.326198135487531412869380069525.dcm#1592617294291742
gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.195975724868

To download the files to the VM filesystem we can use the standard `gsutil` command, which is preinstalled on Colab instances.

In [ ]:
# https://cloud.google.com/storage/docs/gsutil/commands/cp
!cat gcs_paths.txt | gsutil -m cp -I ./downloaded_cohort

BadRequestException: 400 Bucket is requester pays bucket but no user project provided.
CommandException: 1 file/object could not be transferred.


The command above will fail, since IDC-hosted data is stored in US multi-region [requester-pays Storage buckets](https://cloud.google.com/storage/docs/requester-pays). This means that you need to provide a project ID with billing configured to download the data. If you are using Google Colab from the US, the corresponding VM instance will likely be in the US, and data egress charges will be $0.01/GB (see  [GCP network egress charges](https://cloud.google.com/storage/pricing#network-buckets) for full details).

Note that if you want to donwload the data to your own computer, the costs will be much higher.

Before you can refer to a project that you own, you need to authenticate.

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
# https://cloud.google.com/storage/docs/gsutil/commands/cp
!mkdir downloaded_cohort
!cat gcs_paths.txt | gsutil -u my-first-project-289017 -m cp -I ./downloaded_cohort

Copying gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.299257181285137013436464151874/1.2.276.0.7230010.3.1.3.0.27811.1553311167.110292/1.2.276.0.7230010.3.1.4.0.27811.1553311167.110290.dcm#1592618476761048...
Copying gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.224985459390356936417021464571/1.2.276.0.7230010.3.1.3.0.57863.1553343874.661503/1.2.276.0.7230010.3.1.4.0.57863.1553343874.661501.dcm#1592617650034487...
Copying gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.299257181285137013436464151874/1.3.6.1.4.1.14519.5.2.1.6279.6001.251204142823446211001577555181/1.3.6.1.4.1.14519.5.2.1.6279.6001.216241658981991335900886782934.dcm#1592618477049520...
Copying gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.195975724868929317649402600442/1.3.6.1.4.1.14519.5.2.1.6279.6001.229343399861261429237689489892/1.3.6.1.4.1.14519.5.2.1.6279.6001.326198135487531412869380069525.dcm#1592617294291742...
Copying gs://idc-tcia-lidc-idri/

Now the data is located in the file storage local to the VM, but all of the files are in the same directory, which is not the most convenient layout.

You can use the DICOM metadata to organize those instances, or use one of the existing tools to do this. One such tool is used below to organize the flat list of DICOM files into the PatientID-StudyInstanceUID-SeriesInstanceUID-SOPInstanceUID hierarchy.

In [1]:
!git clone https://github.com/pieper/dicomsort.git
!pip install pydicom
!python dicomsort/dicomsort.py --help

Cloning into 'dicomsort'...
remote: Enumerating objects: 126, done.
remote: Total 126 (delta 0), reused 0 (delta 0), pack-reused 126
Receiving objects: 100% (126/126), 37.03 KiB | 1.61 MiB/s, done.
Resolving deltas: 100% (63/63), done.
     |████████████████████████████████| 35.5MB 1.3MB/s 

% dicomsort.py --help
dicomsort [options...] sourceDir targetDir/<patterns>

 where [options...] can be:
    [-z,--compressTargets] - create a .zip file in the target directory
    [-d,--deleteSource] - remove source files/directories after sorting
    [-f,--forceDelete] - remove source without confirmation
    [-k,--keepGoing] - report but ignore dupicate target files
    [-v,--verbose] - print diagnostics while processing
    [-s,--symlink] - create a symlink to dicom files in sourceDir instead of copying them
    [-t,--test] - run the built in self test (requires internet)
    [-u,--unsafe] - do not replace unsafe characters with '_' in the path
    [--help] - print this message

 where sourceDi

The command below will sort instances into folders based on the DICOM metadata stored in the corresponding files.

In [ ]:
!python dicomsort/dicomsort.py -u downloaded_cohort cohort_sorted/%PatientID/%StudyInstanceUID/%SeriesInstanceUID/%SOPInstanceUID.dcm

100% 29/29 [00:02<00:00, 11.31it/s]
Files sorted
